In [0]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# Setting to show all columns
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 4000)
pd.set_option('display.max_colwidth', 1000)

In [0]:
! pip install sportmonks -q
! pip install tzlocal -q

In [0]:
#Day's fixtures
from sportmonks.soccer import SoccerApiV2
soccer = SoccerApiV2(api_token='nEiZYXpxLywA4UWGBcsjCvFhejrrwtkRbrOnw5l0hOkqECMaj3eO7PJB9Pgc')

fixtures = soccer.fixtures_today(includes=('localTeam', 'visitorTeam'))

for f in fixtures:
   print(f['localTeam']['name'], 'plays at home against', f['visitorTeam']['name'])

In [0]:
# All leagues in membership
leagues = soccer.leagues(includes=('Spain'))

for l in leagues:
   print(l)

In [0]:
# All available seasons in membership
# By league_id
all_seasons = soccer.seasons()
all_seasons

In [0]:
# La Liga - Spain
spain_id = 32
laliga_id = 564
laliga_s0 = 13133  # 2018/2019
laliga_s1 = 8442   # 2017/2018
laliga_s2 = 853    # 2016/2017
laliga_s3 = 2063   # 2015/2016
laliga_s4 = 2061   # 2014/2015

# Premier League - England
england_id = 462
premier_id = 8
premier_s0 = 12962 # 2018/2019
premier_s1 = 6397  # 2017/2018
premier_s2 = 13    # 2016/2017
premier_s3 = 10    # 2015/2016
premier_s4 = 12    # 2014/2015

In [0]:
# season results
season_r = soccer.season_results(season_id = 10)
match_count = 0 # Number of matches
comm_count = 0  # Count of matches with commentaries
no_comm = {}    # Info of match w/NO commentaries
for result in season_r:
    if result['commentaries']== True:
        comm_count += 1
    else:
        no_comm = {result['commentaries'], result['id'], result['season_id'],
                    result['round_id'], result['localteam_id'],
                    result['visitorteam_id'], result['venue_id']}
    match_count += 1
    print(result)
    
print(season_r)

In [0]:
print(len(season_r)) # Number of matches
print(match_count)   # Number of matches
print(comm_count)    # Number of matches with commentaries
print(no_comm)       # No commentaries match info

In [0]:
# Commentaries for match
comments_1 = soccer.commentaries(fixture_id = 10367576)
comments_1

### La Liga 2017/2018 Season

#### Results for all matches

In [0]:
# Match Results List - La Liga 2017/2018 season
table_ll_s1 = soccer.season_results(season_id = 8442) # Results La Liga 17/18
lst_ll_s1 = []
for result in table_ll_s1:
    lst_result = [result['id'], result['round_id'], result['localteam_id'],
                  result['visitorteam_id'], result['scores']['localteam_score'],
                  result['scores']['visitorteam_score'], 
                  result['scores']['ft_score'], result['commentaries']]
    lst_ll_s1.append(lst_result)

In [6]:
# Season 1 (2017/18) DataFrame - La Liga
df_laliga_s1 = pd.DataFrame(lst_ll_s1, columns=['fixture_id', 'round_id',
                                       'localteam_id', 'visitorteam_id',
                                       'localteam_score',  'visitorteam_score',
                                       'fulltime_score', 'commentaries'])
df_laliga_s1.head()

,fixture_id,round_id,localteam_id,visitorteam_id,localteam_score,visitorteam_score,fulltime_score,commentaries
0,4193968,131630,3457,3477,1,0,1-0,True
1,4193969,131630,36,594,2,3,2-3,True
2,4193970,131630,13258,106,0,0,0-0,True
3,4193971,131630,231,7980,2,2,2-2,True
4,4193972,131630,676,528,1,1,1-1,True


In [6]:
df_laliga_s1.shape

(380, 8)

#### Shots on Post

In [0]:
# Decisive Goals for Season 1 (2017/18) - La Liga
import re

goal_player = []      # Scoring players
goal_team = []        # Scoring team
goal_score = []       # Updated score after goal
goal_against = []     # Teams scored against
goal_minute = []      # Minute of score
goal_fixture = []       # Fixture id of match's score

# all fixtures from 2017/18 season table
fixtures_lst = df_laliga_s1.fixture_id

# Commentaries for each fixture
for fixture in fixtures_lst:
    commentaries = soccer.commentaries(fixture) 
    
    # Parsing data from commentaries for each fixture
    for dictionary in commentaries:
        
        if dictionary["goal"] == True:        
            # setting 'comment' value to 'comment' variable
            comments = dictionary['comment']
            
            if "Goal!" in comments:
                # 'score' finds digits in 'comment' string and makes digit list
                score = list(map(int, re.findall(r'\d+', comments)))
                goal_score.append(score)
                goal_minute.append(dictionary['minute'])
                goal_fixture.append(fixture)

                # adding scoring player to 'goal_player' 
                half_string = comments.split('.')[1]
                if '-' in half_string:
                    player_string = (half_string.split('-')[0]).strip(' ')
                    goal_player.append(player_string)
                elif '(' in half_string:
                    player_string = half_string.split('(')[0]
                    player_string = player_string.strip(' ')
                    goal_player.append(player_string)

                # adding scoring team to 'goal_team'
                if '-' in half_string:
                    team_string = half_string.split('-')[1]
                    team_string = (team_string.split('-')[0]).strip(' ')
                    goal_team.append(team_string)
                elif '(' in half_string:
                    team_string = half_string.split('(')[1]
                    team_string = (team_string.split(')')[0]).strip(' ')
                    goal_team.append(team_string)
                
                # adding team with goal against
                # will be useful when calculating 'clutch' score
                first_half_string = (comments.split('.')[0]).strip('Goal!')
                teams = re.sub("\d+", " ", first_half_string)
                team_1 = (teams.split(",")[0]).strip(' ')
                team_2 = (teams.split(",")[1]).strip(' ')
                if team_1 == team_string:
                    team_against = team_2
                else:
                    team_against = team_1
                goal_against.append(team_against)
                    



In [0]:
decisive_goals = [goal_player, goal_minute, goal_fixture, goal_team, goal_against, goal_score]

In [0]:
df_decisive_goals = pd.DataFrame(decisive_goals, index=['player', 'minute', 'fixture_id', 'team', 'opponent', 'updated_score']).T

In [59]:
df_decisive_goals.head(10)

,player,minute,fixture_id,team,opponent,updated_score
0,José Luis Morales,88,4193968,Levante,Villarreal,"[1, 0]"
1,Willian José,88,4193969,Real Sociedad,Celta de Vigo,"[2, 3]"
2,Juanmi,80,4193969,Real Sociedad,Celta de Vigo,"[2, 2]"
3,Maxi Gómez,50,4193969,Celta de Vigo,Real Sociedad,"[2, 1]"
4,Mikel Oyarzabal,33,4193969,Real Sociedad,Celta de Vigo,"[1, 1]"
5,Maxi Gómez,22,4193969,Celta de Vigo,Real Sociedad,"[1, 0]"
6,José Giménez,85,4193971,Atlético de Madrid,Girona,"[2, 2]"
7,Ángel Correa,78,4193971,Atlético de Madrid,Girona,"[2, 1]"
8,Christian Stuani,25,4193971,Girona,Atlético de Madrid,"[2, 0]"
9,Christian Stuani,22,4193971,Girona,Atlético de Madrid,"[1, 0]"


In [35]:
df_decisive_goals.isnull().shape

(998, 4)

In [85]:
df_goals_lls1

,comment,extra_minute,fixture_id,goal,important,minute,order
0,"Goal! Levante 1, Villarreal 0. José Luis Morales (Levante) converts the penalty with a right footed shot to the bottom right corner.",NaN,4193968,True,True,88,110.0
1,"Goal! Celta de Vigo 2, Real Sociedad 3. Willian José (Real Sociedad) converts the penalty with a right footed shot to the bottom right corner.",NaN,4193969,True,True,88,95.0
2,"Goal! Celta de Vigo 2, Real Sociedad 2. Juanmi (Real Sociedad) right footed shot from the centre of the box to the bottom right corner. Assisted by Willian José with a through ball.",NaN,4193969,True,True,80,84.0
3,"Goal! Celta de Vigo 2, Real Sociedad 1. Maxi Gómez (Celta de Vigo) right footed shot from the left side of the box to the bottom right corner. Assisted by Gustavo Cabral with a headed pass following a corner.",NaN,4193969,True,True,50,54.0
4,"Goal! Celta de Vigo 1, Real Sociedad 1. Mikel Oyarzabal (Real Sociedad) left footed shot from outside the box to the top left corner.",NaN,4193969,True,True,33,29.0
5,"Goal! Celta de Vigo 1, Real Sociedad 0. Maxi Gómez (Celta de Vigo) right footed shot from the right side of the six yard box to the bottom left corner.",NaN,4193969,True,True,22,15.0
6,"Goal! Girona 2, Atlético de Madrid 2. José Giménez (Atlético de Madrid) header from the centre of the box to the centre of the goal. Assisted by Koke with a cross following a set piece situation.",NaN,4193971,True,True,85,94.0
7,"Goal! Girona 2, Atlético de Madrid 1. Ángel Correa (Atlético de Madrid) right footed shot from outside the box to the top right corner. Assisted by Lucas Hernández.",NaN,4193971,True,True,78,82.0
8,"Goal! Girona 2, Atlético de Madrid 0. Christian Stuani (Girona) header from very close range to the top left corner following a set piece situation.",NaN,4193971,True,True,25,20.0
9,"Goal! Girona 1, Atlético de Madrid 0. Christian Stuani (Girona) header from the centre of the box to the bottom right corner. Assisted by Álex Granell with a cross.",NaN,4193971,True,True,22,17.0


In [0]:
# Decisive Goals for Season 1 (2017/18) - La Liga
goals_lst = [] # 'comment' string - split after '. '

# all fixtures from 2017/18 season table
fixtures_lst1 = df_laliga_s1.fixture_id

# Commentaries for each fixture
for fixture in fixtures_lst:
    commentaries = soccer.commentaries(fixture) 
    # Parsing data from commentaries for each fixture
    for dictionary in commentaries:
        if dictionary["goal"] == True: 
            goals_lst.append(dictionary)

In [0]:
df_goals_lls1 = pd.DataFrame(goals_lst)

In [35]:
[len(each) for each in decisive_goals]

[558, 662, 662, 732, 662]

In [0]:
goal_team

In [70]:
len(df_goals_lls1[df_goals_lls1['comment'].str.contains('Goal! ', regex=False, case=False, na=False)])

998

In [74]:
df_goals_lls1[df_goals_lls1['comment'].str.contains('Levante) converts the penalty with a right footed shot to the bottom right corner.', regex=False, case=False, na=False)]

,comment,extra_minute,fixture_id,goal,important,minute,order
0,"Goal! Levante 1, Villarreal 0. José Luis Mora...",NaN,4193968,True,True,88,110.0


In [0]:
df_goals_lls1 = df_goals_lls1[df_goals_lls1.comment.str.contains("Own Goal") == False]

In [82]:
df_goals_lls1.sh

(998, 7)

#### Decisive Goals

In [0]:
import re
# ----->> TESTING DECISIVE GOALS CODE <<------
#lst_fix_id = df_laliga_s1.fixture_id.tolist()
goal_game = []      # Matches by 'fixture_id'
goal_player = []    # Scoring players
goal_minute = []    # Scoring minute
goal_team = []      # Scoring team
goal_score = []   # 1 if tie broken, 0 for tie
#for fixture in lst_fix_id:
commentaries = soccer.commentaries(fixture_id = 4194048)

for dictionary in commentaries:
    if dictionary["goal"] == True:  
        # setting 'comment' value to 'comment' variable
        comment = dictionary['comment']
        
        if "Goal!" in comment:
            # Finding scores that break tie
            # 'score' finds digits in 'comment' string and makes digit list
            score = list(map(int, re.findall(r'\d+', comment)))
            goal_score.append(score)
            
            # adding scoring player to 'goal_player'    
            player_string = comment.split('. ')[1]
            player_string = (player_string.split(' -')[0]).strip(" ")
            goal_player.append(player_string)
            
            # adding scoring team to 'goal_team'
            if '-' in half_string:
                team_string = half_string.split('-')[1]
                team_string = (player_string.split('-')[0]).strip(" ")
                goal_team.append(team_string)
            elif '(' in half_string:
                team_string = re.search(r"\(([A-Za-z0-9_]+)\)", half_string)
                team_string = player_string.group(1)
                goal_team.append(team_string)


In [13]:
commentaries = soccer.commentaries(fixture_id = 4194048)
for dictionary in commentaries:
    if dictionary["goal"] == True:
        print(dictionary["minute"])

62
58


In [41]:
print(goal_game, goal_team, goal_player, goal_minute, raised_score)

[4194048, 4194048] ['Levante', 'Getafe'] ['José Luis Morales', 'Faycal Fajr'] [62, 58] [0, 1]


In [0]:
fixture_comment= soccer.commentaries(fixture_id = 4194347)
fixture_comment

###  Shots on Post - La Liga Current Season (2018/2019)

In [101]:
# Commentaries for Season 1 (2017/18) - La Liga
lst_fix_id = df_laliga_s0.fixture_id.tolist()
post_lst0 = []
crossbar_lst0 = []
woodwork_lst0 = []
for fixture in lst_fix_id:
    comment= soccer.commentaries(fixture_id = fixture)
    for block in comment:
        if 'post' in block['comment']:
            post_lst0.append(fixture)
        elif 'crossbar' in block['comment']:
            crossbar_lst0.append(fixture)
        elif 'woodwork' in block['comment']:
            woodwork_lst0.append(fixture)

[[10367576, 151245, 13258, 844, 2, 1, '2-1', True], [10367578, 151245, 36, 528, 1, 1, '1-1', True], [10367579, 151245, 60, 274, 1, 2, '1-2', True], [10367580, 151245, 231, 361, 0, 0, '0-0', True], [10367581, 151245, 377, 676, 1, 4, '1-4', True], [10367582, 151245, 83, 2975, 3, 0, '3-0', True], [10367583, 151245, 214, 7980, 1, 1, '1-1', True], [10367584, 151245, 3468, 106, 2, 0, '2-0', True], [10367585, 151245, 3477, 594, 1, 2, '1-2', True], [10367586, 151246, 2975, 485, 0, 0, '0-0', True], [10367587, 151246, 13258, 274, 2, 2, '2-2', True], [10367588, 151246, 7980, 377, 1, 0, '1-0', True], [10367589, 151246, 528, 214, 2, 0, '2-0', True], [10367590, 151246, 231, 3468, 1, 4, '1-4', True], [10367591, 151246, 844, 594, 2, 2, '2-2', True], [10367592, 151246, 676, 3477, 0, 0, '0-0', True], [10367593, 151246, 361, 83, 0, 1, '0-1', True], [10367594, 151246, 3457, 36, 1, 2, '1-2', True], [10367595, 151246, 106, 60, 2, 0, '2-0', True], [10367596, 151247, 2975, 528, 2, 1, '2-1', True], [10367597, 

In [102]:
# Current Season - Season 0 (2018/19) DataFrame - La Liga
df_laliga_s0 = pd.DataFrame(lst_ll_s0, columns=['fixture_id', 'round_id',
                                       'localteam_id', 'visitorteam_id',
                                       'localteam_score',  'visitorteam_score',
                                       'fulltime_score', 'commentaries'])
df_laliga_s0.head()

,fixture_id,round_id,localteam_id,visitorteam_id,localteam_score,visitorteam_score,fulltime_score,commentaries
0,10367576,151245,13258,844,2,1,2-1,True
1,10367578,151245,36,528,1,1,1-1,True
2,10367579,151245,60,274,1,2,1-2,True
3,10367580,151245,231,361,0,0,0-0,True
4,10367581,151245,377,676,1,4,1-4,True


In [104]:
df_laliga_s0.shape

(149, 8)

In [0]:
# Commentaries for Season 1 (2017/18) - La Liga
lst0_fix_id = df_laliga_s0.fixture_id.tolist()
post_lst0 = []
crossbar_lst0 = []
woodwork_lst0 = []
for fixture in lst0_fix_id:
    comment= soccer.commentaries(fixture_id = fixture)
    for block in comment:
        if 'post' in block['comment']:
            post_lst0.append(fixture)
        elif 'crossbar' in block['comment']:
            crossbar_lst0.append(fixture)
        elif 'woodwork' in block['comment']:
            woodwork_lst0.append(fixture)

In [121]:
print(post_lst0)
print('"post" hit count:', len(post_lst0))
print(crossbar_lst0)
print(woodwork_lst0)

[10367582, 10367583, 10367584, 10367587, 10367587, 10367592, 10367600, 10367602, 10367609, 10367610, 10420458, 10420463, 10420469, 10420470, 10420473, 10420476, 10420478, 10420484, 10420484, 10420493, 10420494, 10420504, 10420507, 10420508, 10420510, 10420510, 10420516, 10420517, 10420517, 10420518, 10420518, 10420525, 10420532, 10420533, 10420534, 10420535, 10420549, 10420550, 10420550, 10420551, 10420551, 10420553, 10420553]
"post" hit count: 43
[]
[]


#### VAR in La Liga - Current Season (2018/2019)

In [0]:
# Commentaries for Current Season - Season 0 (2018/19) - La Liga
s0_fix_id = df_laliga_s0.fixture_id.tolist()
var_lst = []
review_lst = []
for fixture in s0_fix_id:
    comment= soccer.commentaries(fixture_id = fixture)
    for block in comment:
        if 'VAR' in block['comment']:
            var_lst.append(fixture)
        if 'Video Review' in block['comment']:
            review_lst.append(fixture)

In [112]:
print(var_lst)
print('"VAR" comment count:', len(var_lst))
print(review_lst)
print('"Video Review" comment count:', len(review_lst))

[10367581, 10367581, 10367589, 10367590, 10367596, 10367596, 10367597, 10367602, 10367604, 10367605, 10367609, 10420444, 10420449, 10420452, 10420456, 10420463, 10420471, 10420472, 10420472, 10420473, 10420475, 10420481, 10420482, 10420482, 10420483, 10420487, 10420489, 10420492, 10420493, 10420493, 10420494, 10420502, 10420502, 10420507, 10420507, 10420507, 10420507, 10420512, 10420515, 10420517, 10420521, 10420526, 10420528, 10420528, 10420533, 10420539, 10420539, 10420540, 10420545, 10420550]
"VAR" comment count: 50
[10367581, 10367581, 10367589, 10367590, 10367597, 10367602, 10367603, 10367604, 10367605, 10367609, 10420444, 10420449, 10420452, 10420456, 10420463, 10420471, 10420472, 10420472, 10420473, 10420475, 10420481, 10420482, 10420482, 10420483, 10420487, 10420492, 10420493, 10420493, 10420494, 10420502, 10420502, 10420507, 10420507, 10420507, 10420512, 10420515, 10420517, 10420521, 10420526, 10420528, 10420528, 10420533, 10420539, 10420539, 10420540, 10420545, 10420550]
"Vid

In [0]:
# Checking VAR comments in current season
fixture_comment= soccer.commentaries(fixture_id = 10367596)
fixture_comment